<a href="https://colab.research.google.com/github/jonmichaelstroh/Krill-Mania/blob/main/Master_Collab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ipyleaflet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.3 MB 15.3 MB/s 


In [ ]:
#packages
import pandas as pd
import numpy as np
import ipyleaflet
from ipyleaflet import (
    Map,
    basemaps,
    basemap_to_tiles,
    projections,
    GeoData, 
    Heatmap,
    Circle,
    GeoJSON,
    AntPath,
    Polyline,
    Marker
)
from ipywidgets import Layout
import json
from pandas.io.json import json_normalize
from ipywidgets import HTML 
from ipyleaflet import Popup 
from datetime import datetime 

In [ ]:
geo = GeoJSON(data=uploaded)

NameError: ignored

In [ ]:
#our antartic map
spsLayout=Layout(width='800px', height='800px')

amap = Map(center=(-70, -80),
        zoom=2,
        layout=spsLayout,
        basemap=basemaps.NASAGIBS.BlueMarble3031,
        crs=projections.EPSG3031)

heatmap = Heatmap(
    locations=geo,
    radius=20)

amap.add_layer(heatmap)
amap

NameError: ignored

I'm going to put in the boat visual data and code here -->


In [ ]:
#Read in lat/long data
latlon = pd.concat(map(pd.read_csv, ['/content/lg2004_364_72000_nb150_latlon.csv', '/content/lg2004_365_21600_nb150_latlon.csv', '/content/lg2004_365_36000_nb150_latlon.csv','/content/lg2005_000_50400_nb150_latlon.csv', '/content/lg2005_001_50400_nb150_latlon.csv', '/content/lg2005_002_14400_nb150_latlon.csv', '/content/lg2005_003_63255_nb150_latlon.csv', '/content/lg2005_004_07200_nb150_latlon.csv', '/content/lg2005_005_07200_nb150_latlon.csv', '/content/lg2005_006_79200_nb150_latlon.csv', '/content/lg2005_007_21600_nb150_latlon.csv', '/content/lg2005_008_07200_nb150_latlon.csv', '/content/lg2005_009_14400_nb150_latlon.csv', '/content/lg2005_010_21600_nb150_latlon.csv', '/content/lg2005_011_14400_nb150_latlon.csv', '/content/lg2005_012_14400_nb150_latlon.csv', '/content/lg2005_013_21600_nb150_latlon.csv', '/content/lg2005_014_21600_nb150_latlon.csv', '/content/lg2005_015_21600_nb150_latlon.csv', '/content/lg2005_016_07200_nb150_latlon.csv', '/content/lg2005_017_14400_nb150_latlon.csv', '/content/lg2005_018_07200_nb150_latlon.csv', '/content/lg2005_019_21600_nb150_latlon.csv', '/content/lg2005_020_14400_nb150_latlon.csv', '/content/lg2005_021_21600_nb150_latlon.csv', '/content/lg2005_022_57600_nb150_latlon.csv', '/content/lg2005_023_36000_nb150_latlon.csv', '/content/lg2005_024_50400_nb150_latlon.csv', '/content/lg2005_025_28800_nb150_latlon.csv', '/content/lg2005_026_43200_nb150_latlon.csv', '/content/lg2005_028_60274_nb150_latlon.csv', '/content/lg2005_029_08119_nb150_latlon.csv', '/content/lg2005_030_07200_nb150_latlon.csv', '/content/lg2005_031_11629_nb150_latlon.csv', '/content/lg2005_032_14400_nb150_latlon.csv', '/content/lg2005_033_28800_nb150_latlon.csv', '/content/lg2005_034_07200_nb150_latlon.csv', '/content/lg2005_035_07200_nb150_latlon.csv']))
#I put in tons...to find a midpoint you can adjust this and start with one
latlon = latlon.iloc[::1000, :] #take every 1000 data point (for runtime)

#format change for locations
locations = []
for i in range(len(latlon)):
  locations.append([latlon.iloc[i][0], latlon.iloc[i][1]])

In [ ]:
#antmap
spsLayout=Layout(width='800px', height='800px')

amap = Map(center=(-70, -80),
        zoom=2,
        layout=spsLayout,
        basemap=basemaps.NASAGIBS.BlueMarble3031,
        crs=projections.EPSG3031)

ant_path = AntPath(
    locations= locations)

amap.add_layer(ant_path)
marker = Marker(location=(latlon.iloc[0][0], latlon.iloc[0][1]), draggable=False)
amap.add_layer(marker) #I added some markers for the first couple stops along the journey (Day 364, 365, 0 I believe), but they are not 
#interactive, and they are at the first point of each day (which may be easier than the midpoint) but they way I did it is cumbersome so there must be something better
marker2 = Marker(location=(latlon.iloc[1][0], latlon.iloc[1][1]), draggable=False)
amap.add_layer(marker2)
marker3 = Marker(location=(latlon.iloc[2][0], latlon.iloc[2][1]), draggable=False)
amap.add_layer(marker3)
amap




Map(center=[-70, -80], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [ ]:
#find median locations from each dataset 

import os 
import statistics 

loclist = []
medlocs = []
for listname in os.listdir(): 
  n = listname.split("_") 
  if len(n) == 5:
    if n[4] == 'latlon.csv': #find latlon files
      loclist.append(listname)
      current_lats = []
      current_longs = []
      current_year = n[0][2:]
      current_day = n[1] 
      current_byt = n[2]
      with open(listname, 'r') as f: #open file
        file = f.readlines() 
        for pt in file[1:]: 
          pt = ' '.join(pt.splitlines())
          try:
            lat = float(pt.split(",")[0])
            long = float(pt.split(",")[1])
            current_lats.append(lat)
            current_longs.append(long)
          except ValueError:
            pass
        try:
          medlat = statistics.median(current_lats)
          medlong = statistics.median(current_longs)
        except statistics.StatisticsError:
          pass
      medlocs.append((medlat, medlong, current_year, current_day, current_byt))

  
  
  




'''for path in os.listdir()[0:2]: 
  if len(path.split('_')) == 5: 
    with open(path, 'r') as f: 
      file = f.readlines() 
      print(file)'''

'''for loc in allfiles: 
  medlat = statistics.median(loc) 
  medlong = statistics.median(long)
  medlocs.append((medlat,medlong))

print(medlocs)'''





'for loc in allfiles: \n  medlat = statistics.median(loc) \n  medlong = statistics.median(long)\n  medlocs.append((medlat,medlong))\n\nprint(medlocs)'

In [ ]:
#add median locations as markers on map 
for loc in medlocs: 
  marker = Marker(location = (loc[0], loc[1]), draggable = False)
  daynum = str(int(loc[3]) + 1)
  daynum.rjust(3 + len(daynum), '0') 
  year = str(int(loc[2]))
  res = datetime.strptime(year + "-" + daynum, "%Y-%j").strftime("%m-%d-%Y")
  date = HTML() 
  date.value = "date: " + str(res)
  marker.popup = date 
  amap.add_layer(marker)
amap

Map(bottom=1244.740234375, center=[-67.72085106261056, -72.56391883999814], controls=(ZoomControl(options=['po…